In [ ]:
%pip install lxml

In [ ]:
import os
from lxml import etree

In [ ]:
# Путь к папке с XLIFF файлами. Для Google Colab по умолчанию '/content/'
path = '/content/'

In [ ]:
# Список всех XLIFF файлов в указанной папке
files = os.listdir(path)
files_list = [os.path.join(path, f) for f in files if os.path.splitext(f)[1] == '.xliff']
files_list

In [ ]:
# Проходим по каждому файлу в списке
for input_file in files_list:
    # Загружаем XML файл с использованием lxml
    try:
        tree = etree.parse(input_file)
        root = tree.getroot()
        # print(f"Tree: {tree}")
        # print(f"Root: {root}")
    except etree.XMLSyntaxError as e:
        print(f"Error parsing \"{os.path.basename(input_file)}\": {e}")
        continue
    
    namespaces = root.nsmap # Получаем словарь пространств имен из корня
    # print(f"Namespace: {namespaces}")

    # Находим элемент <file> и меняем source-language на target-language
    file_elem = root.find('ns:file', namespaces={'ns': namespaces[None]})
    if file_elem is not None:
        source_lang = file_elem.attrib.get('source-language')
        target_lang = file_elem.attrib.get('target-language')
        if source_lang and target_lang:
            file_elem.attrib['source-language'] = target_lang
    
    # Применяем это пространство имен в XPath
    trans_units = root.xpath('//ns:body/ns:trans-unit', namespaces={'ns': namespaces[None]})

    # Проходим по всем тегам <trans-unit>
    for trans_unit in trans_units:
        source = trans_unit.find('ns:source', namespaces={'ns': namespaces[None]})
        target = trans_unit.find('ns:target', namespaces={'ns': namespaces[None]})
        
        # Если элементы source и target найдены
        if source is not None and target is not None:
            # Переносим текст из target в source
            source.text = target.text

            # Меняем состояние в target с 'final' на 'needs-translation'
            if 'state' in target.attrib and target.attrib['state'] != 'needs-translation':
                target.attrib['state'] = 'needs-translation'
    
    # Сохраняем изменённый файл в исходном формате, сохраняя декларацию XML и пространство имен
    output_file = os.path.join(path, f"output_{os.path.basename(input_file)}")
    tree.write(output_file, encoding=tree.docinfo.encoding if tree.docinfo.encoding else "utf-8", xml_declaration=True, pretty_print=True)
    print(f"\"{os.path.basename(input_file)}\": Completed")